# import¶

In [1]:
# include working dir
!conda install -y gdown
!gdown --id 1xB5Ec_OkjxhE7q2Iw4t6REBlLto1O2Qx
# !gdown --id 1VCbWE-DyVYZdDViH75ts62lrkr3rBe_V

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - rapidsai/linux-64::libcuml==21.12.00=cuda11_g04c4927f3_0
  - conda-forge/linux-64::abseil-cpp==20211102.0=h93e1e8c_3
  - rapidsai/linux-64::dask-cudf==21.12.02=cuda_11_py37_g06540b9b37_0
  - conda-forge/linux-64::pyarrow==5.0.0=py37h8cf84b7_35_cuda
  - rapidsai/linux-64::cuml==21.12.00=cuda11_py37_g04c4927f3_0
  - conda-forge/linux-64::grpc-cpp==1.45.2=he70e3f0_3
  - rapidsai/linux-64::libcudf==21.12.02=cuda11_g06540b9b37_0
  - conda-forge/linux-64::arrow-cpp==5.0.0=py37h846d386_35_cuda
  - rapidsai/linux-64::cudf==21.12.02=cuda_11_py37_g06540b9b37_0
  - conda-forge/noarch::parquet-cpp==1.5.1=2
  - conda-forge/linux-64::libabseil==20211102.0=cxx17_h48a1fff_3
done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.layers import (Input, Conv1D, BatchNormalization, MaxPooling1D, 
                          AveragePooling1D, Flatten, Dense,Add, Activation, 
                          ZeroPadding1D, GlobalMaxPooling1D,MaxPool1D)
from keras.layers import Reshape, Permute, Concatenate, Dropout 
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,CSVLogger, EarlyStopping)
from sklearn.model_selection import KFold 
import tensorflow as tf
from tensorflow import keras

In [3]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [4]:
# load dataset
df = pd.read_csv('/kaggle/working/dataset_C.zip')

In [5]:
X = df.drop(columns='output')
y = df['output']

In [6]:
# encode class values as integers
y = pd.get_dummies(y, ['output'])
# Convert all columns to floats
y = y.astype(float)

In [7]:
def dataSplit(trainval_idx, test_idx):
    X_train = X.iloc[trainval_idx,:]
    y_train = y.iloc[trainval_idx,]
    X_training = X_train.iloc[:int(0.8*len(trainval_idx)), :]
    X_val = X_train.iloc[int(0.8*len(trainval_idx)):, :]
    y_training = y_train.iloc[:int(0.8*len(trainval_idx)),]
    y_val = y_train.iloc[int(0.8*len(trainval_idx)):,]

    X_test = X.iloc[test_idx,:]
    y_test = y.iloc[test_idx,]

    return X_training, X_val, y_training, y_val, X_test, y_test

# Baseline Model

## CR-Former model Architectuer

In [12]:
def identity_block(X, filters, stage, block):
    conv_name_base = f"res{stage}{block}_branch"
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2 = filters
    X_shortcut = X
    X = Conv1D(F1, 1, strides=1, padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    X = Conv1D(F2, 1, strides=1, padding='valid', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2b')(X)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

def convolutional_block(X, filters, stage, block, s=1):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2 = filters
    X_shortcut = X
    X = Conv1D(filters=F1, kernel_size=1, strides=s, padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    X = Conv1D(filters=F2, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2b')(X)
    X_shortcut = Conv1D(filters=F2, kernel_size=1, strides=s, padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=1, name=bn_name_base + '1')(X_shortcut)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

def ResNet_Block(X, f_size, f_num, convblock, i1block, i2block):
#     #     stage 0
#     X = Conv1D(f_num,f_size , strides=1, name='convres', kernel_initializer=glorot_uniform(seed=0))(X)
#     X = MaxPooling1D(pool_size = 3, strides=2)(X)
    # stage 1-7
    for stage in range(1, 8):
        if stage%2==0:
            X = convolutional_block(X, filters=[f_num, f_num], stage=stage, block=convblock)
        else:
            X = convolutional_block(X, filters=[f_num, f_num], stage=stage, block=convblock)
            X = identity_block(X, [f_num, f_num], stage=stage, block=i1block)
            X = identity_block(X, [f_num, f_num], stage=stage, block=i2block)
    # final stage
#     X = AveragePooling1D(pool_size=2, padding='same')(X)
    return X

def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate=0.1):
    x = LayerNormalization()(inputs)
    x = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(x, x)
    x = Dropout(dropout_rate)(x)
    x = Add()([inputs, x])
    x = LayerNormalization()(x)
    x = Dense(ff_dim, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(inputs.shape[-1])(x)
    x = Dropout(dropout_rate)(x)
    x = Add()([inputs, x])
    return x

def build_model(input_shape): 
    input_layer = Input(shape=input_shape)
    
    # Layer 1: Conv1D
    layer = Conv1D(filters=4, kernel_size=16, strides=1, padding='valid', kernel_initializer=glorot_uniform(seed=0))(input_layer)
    layer = BatchNormalization()(layer)

    # Layer 2: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)

    # Layer 3: Conv1D
    layer = Conv1D(filters=6, kernel_size=8, strides=1, padding='valid', kernel_initializer=glorot_uniform(seed=0))(layer)
    layer = BatchNormalization()(layer)

    # Layer 4: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)
    
    # For Skip Connection
    X_shortcut = layer
    
    # Layer 5: ResNet1
    layer = ResNet_Block(layer, 8, 8,'a','b','c')

    # Layer 6: ResNet2
    layer = ResNet_Block(layer, 16, 10,'d','e','f')
    
    # Reshaping And Padding For Additon And Skip Connection
    X_shortcut=Reshape([6,242])(X_shortcut)
    pad = ZeroPadding1D(padding=((4,0)))(X_shortcut)
    X_shortcut=Reshape([242,10])(pad)
    layer = Add()([layer, X_shortcut])

    # Layer 7: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)
    
    # For Skip Connection
    X_shortcut = layer

    # Layer 8: ResNet3
    layer = ResNet_Block(layer,16,12,'g','h','i') 

    # Reshaping And Padding For Additon And Skip Connection
    X_shortcut=Reshape([10,121])(X_shortcut)
    pad = ZeroPadding1D(padding=((2,0)))(X_shortcut)
    X_shortcut=Reshape([121,12])(pad)
    layer = Add()([layer, X_shortcut])

    # Layer 9: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)

    # For Skip Connection
    X_shortcut = layer

    # Layer 10: ResNet4
    layer = ResNet_Block(layer,4,14 ,'j','k','n')
    layer = MaxPooling1D(pool_size = 2, strides=2)(layer)   
    X_shortcut=Reshape([12,60])(X_shortcut)
    pad = ZeroPadding1D(padding=((2,0)))(X_shortcut)
    X_shortcut=Reshape([60,14])(pad)
    X_shortcut = MaxPooling1D(pool_size = 2, strides=2)(X_shortcut)
    layer = Add()([layer, X_shortcut])

    # Layer 11: AveragePooling1D
    layer = AveragePooling1D(pool_size=2, strides=1, padding='same')(layer)

    # Layer 12: Reshape
    #layer = Reshape((420,))(layer)

    # Layer 13-15: Transformer encoding
    num_heads = 5
    feed_forward_dim = 248
    for _ in range(3):
        layer = transformer_encoder(layer, num_heads, feed_forward_dim)
 
    # Layer 16: Fully connected
    layer = Flatten()(layer)
    layer = Dense(units=248, activation='relu')(layer)

    # Layer 17: Softmax
    output_layer = Dense(units=4, activation='softmax')(layer)

    model = Model(inputs=input_layer, outputs=output_layer, name='CRFormer')
#     model.summary()
    return model

## Train Model & Set parameter

In [13]:
# Optimization settings
num_samples = X.shape[1]
num_features = 1
lr = 2 * 10**-1
batch_size = 50
num_epochs = 20
opt = Adam(lr)

# define a callback functions
def getCallbacks(fold_index):
    checkpoint = ModelCheckpoint(
        filepath='model.h5', # file path for saving the model
        monitor='val_accuracy', # quantity to monitor
        save_best_only=True, # save only the best model (based on the monitor quantity)
        save_weights_only=False, # save the entire model (not just the weights)
        mode='auto', # auto: the mode is inferred from the monitor quantity
        verbose=1 # print messages about the saving process
    )
    early_stopping = EarlyStopping(
        monitor='val_accuracy', # quantity to monitor
        patience=3, # number of epochs with no improvement before stopping
        verbose=1 # print messages about the early stopping process
    ) 

    return [checkpoint, early_stopping]

model = build_model(input_shape=(num_samples, num_features))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
# # Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# # Initialize list to store accuracy scores and models
val_acc_list = []
test_acc_list = []
models = []
fold_index = 0

In [15]:
i = 0
# # Loop through the K-fold splits
for trainval_idx, test_idx in kfold.split(X, y):
    # split data to train val test
    X_training, X_val, y_training, y_val, X_test, y_test = dataSplit(trainval_idx, test_idx)

    # Define file name with fold index
    callbacks = getCallbacks(fold_index)
    fold_index += 1
    
    if i>6:
        # Fit the model on the training data
        model.fit(X_training, y_training, epochs=num_epochs, batch_size=batch_size,
        callbacks=callbacks, validation_data=(X_val, y_val))

        model = load_model('model.h5')
    elif i==6:
        model = load_model(f'/kaggle/input/models2/model (6).h5')
    else:
        model = load_model(f'/kaggle/input/models/model ({i}).h5')

    val_acc = model.evaluate(X_val, y_val)

    # Append the accuracy score and model to the list of scores and models
    val_acc_list.append(val_acc[1] * 100)
    
    # Step 6: repeat for all folds and calculate overall performance metrics
    test_acc = model.evaluate(X_test, y_test)

    # Append the accuracy score and model to the list of scores and models
    test_acc_list.append(test_acc[1] * 100)
    
    print(f'Validation accuracy: {val_acc[1]:.4f} | Test accuracy: {test_acc[1]:.4f}')

    models.append(model)
    
    i+=1


250/250 [==============================] - 8s 32ms/step - loss: 0.6603 - accuracy: 0.9131
Validation accuracy: 0.5798 | Test accuracy: 0.9131
250/250 [==============================] - 8s 31ms/step - loss: 0.5485 - accuracy: 0.9119
Validation accuracy: 0.5815 | Test accuracy: 0.9119
250/250 [==============================] - 8s 30ms/step - loss: 0.6269 - accuracy: 0.8957
Validation accuracy: 0.4658 | Test accuracy: 0.8957
250/250 [==============================] - 8s 31ms/step - loss: 0.1193 - accuracy: 0.9542
Validation accuracy: 0.7800 | Test accuracy: 0.9542
250/250 [==============================] - 8s 31ms/step - loss: 0.5633 - accuracy: 0.8791
Validation accuracy: 0.4267 | Test accuracy: 0.8791
250/250 [==============================] - 8s 34ms/step - loss: 0.4215 - accuracy: 0.9126
Validation accuracy: 0.5696 | Test accuracy: 0.9126
250/250 [==============================] - 7s 30ms/step - loss: 0.2099 - accuracy: 0.9409
Validation accuracy: 0.7031 | Test accuracy: 0.9409
Epoch 

In [16]:
# Step 7: calculate overall performance metrics by averaging the results from all folds
avg_val_acc = np.mean(val_acc_list)
avg_test_acc = np.mean(test_acc_list)
print(f'Overall validation accuracy: {avg_val_acc:.4f} | Overall test accuracy: {avg_test_acc:.4f}')

# Save the best model val
val_best_model_index = np.argmax(val_acc_list)
val_best_model = models[val_best_model_index]
val_best_model.save("val_best_model.h5")

# Save the best model test
test_best_model_index = np.argmax(test_acc_list)
test_best_model = models[test_best_model_index]
test_best_model.save("test_best_model.h5")

Overall validation accuracy: 60.2951 | Overall test accuracy: 91.9037


In [21]:
val_acc_list

[57.97916650772095,
 58.145833015441895,
 46.58333361148834,
 77.99999713897705,
 42.66666769981384,
 56.958335638046265,
 70.3125,
 58.069443702697754,
 63.104164600372314,
 71.13194465637207]

In [22]:
test_acc_list

[91.31249785423279,
 91.18750095367432,
 89.57499861717224,
 95.42499780654907,
 87.91249990463257,
 91.26250147819519,
 94.0874993801117,
 91.39999747276306,
 92.65000224113464,
 94.22500133514404]

## Visualize Model Training History

In [17]:
best_model = load_model('/kaggle/working/test_best_model.h5')
print(best_model.summary())

Model: "CRFormer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1000, 1)]    0           []                               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 985, 4)       68          ['input_2[0][0]']                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 985, 4)      16          ['conv1d_2[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 max_pooling1d_6 (MaxPooling1D)  (None, 492, 4)      0           ['batch_normalization_2[0]

## Save Model without_callbacks

In [18]:
model.save("my_model_without_callbacks.h5")
# best_model.save("my_model_without_callbacks.tflite")
# tensorflow_graph = tf.saved_model.load("/kaggle/working/my_model_without_callbacks.h5")
# config = model.get_config()  # Retrieve the config
# At loading time, register the custom objects with a `custom_object_scope`:
# custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
# with keras.utils.custom_object_scope(custom_objects):
#     new_model = keras.Model.from_config(config)
# serialize model to JSON
model_json = model.to_json()
with open("ECG_classification_model_interpatient_without_callbacks.json", "w") as json_file:
    json_file.write(model_json)
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('interpatient_model_without_callbacks.tflite', 'wb') as f:
    f.write(tflite_model)

print('TFLite Model save done!')

TFLite Model save done!


## Evaluate Model

In [19]:
best_model = load_model('/kaggle/working/model.h5')
accuracy_score(y_test, best_model.predict(X_test).round())

250/250 [==============================] - 15s 30ms/step


0.94225

## Communication using case study

In [23]:
# test_model
# test_case = pd.read_csv("/kaggle/input/testdata/Book2.csv")
pred = best_model.predict(X_test.head(1)).round()
out_col = ['output_CAD', 'output_CHF', 'output_MI', 'output_Normal']
output = {out_col[i]:pred[0][i] for i in range(4)}
output_class = max(output, key=output.get).split('_')[-1]
"output_class = " + output_class

1/1 [==============================] - 7s 7s/step


'output_class = Normal'

# End Model Training Notebook